In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My\ Drive


/content/drive/My Drive


In [0]:
import tensorflow as tf
from tensorflow.keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [10]:
from tensorflow.keras.models import load_model
model = load_model('weights-improvement.hdf5')
print(model.outputs)
print(model.inputs)

[<tf.Tensor 'dense_2/Identity:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(None, 229) dtype=float32>, <tf.Tensor 'input_2:0' shape=(None, 229) dtype=float32>, <tf.Tensor 'input_3:0' shape=(None, 229) dtype=float32>]


In [0]:
import pickle
from keras.preprocessing.text import  tokenizer_from_json

In [0]:
with open("tokens.pickle","rb") as f:
    tokens = pickle.load(f)
    # tokenizer = tokenizer_from_json(data)

In [0]:
import pickle
from keras.preprocessing.sequence import pad_sequences

def encode_text(tokenizer, lines, length):
  # integer encode
  encoded = tokenizer.texts_to_sequences(lines)
     # pad encoded sequences
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [0]:
# sentense = "I love this product. it is the best "
sentense = "i love you but as friend"

X_new =  encode_text(tokens, [sentense], 229)

In [42]:

ynew = model.predict([X_new,X_new,X_new])
if ynew[0][0]<0.5: 
  prediction = 'Negative'
else :
  prediction = 'Positive'  
# show the inputs and predicted outputs
print(f"sentense: {sentense} \nPredicted as: {prediction}" )

sentense: i love you but as friend 
Predicted as: Positive


In [43]:
ynew[0][0]

0.8463756

## kumat tflite

In [0]:
!pip install --upgrade tensorflow

In [0]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_name = "/content/mymodel.tflite"
open(tflite_model_name, "wb").write(tflite_model)

In [44]:
import numpy as np

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/mymodel.tflite") 
interpreter.allocate_tensors()
# Get input and output tensors
input_details = interpreter.get_input_details() 
output_details = interpreter.get_output_details()

# Test model on random input data
x_new= X_new.astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], x_new)
interpreter.set_tensor(input_details[1]['index'], x_new)
interpreter.set_tensor(input_details[2]['index'], x_new)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index']) 
print(output_data)

[[0.8463756]]


## Kumar pb

In [0]:
import tensorflow as tf

In [0]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
from keras import backend as K


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

tf.train.write_graph(frozen_graph, "/content/", "my_model.pb", as_text=False)

In [16]:
from tensorflow.python.platform import gfile
import tensorflow as tf
with tf.Session() as sess:
    # load model from pb file

    # model = load_model('weights-improvement.hdf5')
    with gfile.FastGFile('/content/my_model.pb','rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    # model = load_model('weights-improvement.hdf5')
    
    # model.predict([X_new,X_new,X_new])
    # inference by the model (op name must comes with :0 to specify the index of its output)
    tensor_output = sess.graph.get_tensor_by_name('import/dense_2/Sigmoid:0')
    tensor_input1 = sess.graph.get_tensor_by_name('import/input_1:0')
    tensor_input2 = sess.graph.get_tensor_by_name('import/input_2:0')
    tensor_input3 = sess.graph.get_tensor_by_name('import/input_3:0')
    predictions = sess.run(tensor_output, {tensor_input1: X_new,
                                           tensor_input2: X_new,
                                           tensor_input3: X_new})
    # model.predict([X_new,X_new,X_new])

    print('\n===== output predicted results =====\n')
    print(predictions)


===== output predicted results =====

[[0.33032504]]


## Brian

In [0]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 285 variables.


In [0]:
tf.train.write_graph(frozen_graph, "/model/weights-improvement.hdf5", "/model/sentiments_model.pb", as_text=False)

'/model/sentiments_model.pb'

In [0]:
import tensorflow as tf
from tensorflow.python.platform import gfile

f = gfile.FastGFile("/model/sentiments_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

sess = tf.Session()
sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

In [0]:
# [<tf.Tensor 'dense_2_1/Sigmoid:0' shape=(?, 1) dtype=float32>]
# [<tf.Tensor 'input_1_1:0' shape=(?, 229) dtype=float32>, <tf.Tensor 'input_2_1:0' shape=(?, 229) dtype=float32>, <tf.Tensor 'input_3_1:0' shape=(?, 229) dtype=float32>]

In [0]:
softmax_tensor = sess.graph.get_tensor_by_name('import/dense_2_1/Sigmoid:0')


KeyError: ignored

In [0]:

# [<tf.Tensor 'dense_2/Sigmoid:0' shape=(?, 1) dtype=float32>]
# [<tf.Tensor 'input_1:0' shape=(?, 229) dtype=float32>, <tf.Tensor 'input_2:0' shape=(?, 229) dtype=float32>, <tf.Tensor 'input_3:0' shape=(?, 229) dtype=float32>]

In [0]:
predicts_1 = sess.run(softmax_tensor, {
    'import/input_1_1:0': X[:20], 
    'import/input_2_1:0': X[:20],
    'import/input_3_1:0': X[:20]})

NameError: ignored

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import tensorflow as tf
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, encoding='utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
import tensorflow as tf
# Show current session graph with TensorBoard in Jupyter Notebook.
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
tf.train.write_graph(tf.get_default_graph().as_graph_def(), '/model/',
                     'saved_model.pb', as_text=False)

In [0]:
model.predict("")

In [0]:
import pandas as pd
df = pd.read_csv('amazon.csv')
df = df[['text','label']]
df.head()

,text,label
0,"Reference Yes, Learning or Instruction No: I p...",0
1,NO!!!!!!I will give it 1 star.: This doll is n...,0
2,Neat Features/Uncomfortable: I bought this wit...,0
3,Progressive-Underground divas??? where's the v...,1
4,The theif who stole a week.: What a stinker! I...,0


In [0]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Conv1D,Activation,Dense,Flatten,Input,Dropout,MaxPooling1D,concatenate
from keras.models import Model,Sequential
from keras.layers.embeddings import Embedding

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
# encode a list of lines
import pickle
def encode_text(tokenizer, lines, length):
  # integer encode
  encoded = tokenizer.texts_to_sequences(lines)
     # pad encoded sequences
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded


In [0]:
import pickle
from keras.preprocessing.text import  tokenizer_from_json


In [0]:
with open("tokens.pickle","rb") as f:
    tokens = pickle.load(f)

In [0]:
X =  encode_text(tokens, df['text'][500:800], 229)

In [0]:
X

array([[  397,  2413, 18999, ...,     0,     0,     0],
       [  264,  1041,   905, ...,     0,     0,     0],
       [  547,   801,   963, ...,     0,     0,     0],
       ...,
       [  352,  8065,     1, ...,     0,     0,     0],
       [  453,  4120,   801, ...,     0,     0,     0],
       [ 2643,   585,  1226, ...,     0,     0,     0]], dtype=int32)

In [0]:
from sklearn.model_selection import  train_test_split
X = df['text']
y = [i for i in df['label']]
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify=y)